### IMPORTS ###

In [1]:
from collections import deque
import copy
import random
from multiprocessing import cpu_count, Pool

 ### CLASSES ###

In [2]:
class Sequence_Node:
    def __init__(self, seq, id, pos):
        self.seq = seq
        self.id = id
        self.pos = pos

In [3]:
class Sequence_Node_Set:
    def __init__(self, id):
        self.id = id
        self.members = set()
        
    def add_member(self, member):
        self.members.add(member)

In [4]:
class Reference_Pair:
    def __init__(self, pair_id, seq1, seq2, pos1, pos2, num_seq):
        self.pair_id = pair_id
        self.seq1 = seq1
        self.seq2 = seq2
        self.pos1 = pos1
        self.pos2 = pos2
        
        self.node_sets = [] 
        for i in range(2, num_seq):
            self.node_sets.append(Sequence_Node_Set(i))
            
        self.tree_list = []

        
    

In [5]:
class Tree_Node:
    def __init__(self, seq, seq_id, pos, depth=0):
        self.seq = seq
        self.seq_id = seq_id
        self.pos = pos
        self.depth = depth
        
        self.children = []
        self.parent = None

    def add_child(self, child_node):
        child_node.parent = self
        child_node.depth = self.depth + 1
        self.children.append(child_node)
        

    def remove_child(self, child_node):
        if child_node in self.children:
            child_node.parent = None
            self.children.remove(child_node)

In [6]:
class Tree:
    def __init__(self, root):
        self.root = root
        self.current_leaves = []
        self.height = 0
        self.id = 0
    
    def save_leaf(self, leaf):
        self.current_leaves.append(leaf)
        
    def unsave_leaf(self, leaf):
        if leaf in self.current_leaves:
            self.current_leaves.remove(leaf)

In [7]:
class Clique:
    def __init__(self, path):
        self.nodes = path
        self.seqs = []
        for node in self.nodes:
            self.seqs.append(node.seq)
            
        self.consensus = ""
        self.score = -1
        self.rank = -1
            

### FUNCTIONS ###

In [8]:
def read_file_to_list(filename):
    try:
        # Open the file in read mode
        with open(filename, 'r') as file:
            # Read all lines from the file and store them in a list
            lines = file.readlines()
        # Remove newline characters from each line and strip any leading/trailing whitespaces
        lines = [line.strip() for line in lines]
        
        return lines
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return []
    except Exception as e:
        print(f"Error: {e}")
        return []

In [9]:
def hamming_distance(str1, str2):

    return sum(bit1 != bit2 for bit1, bit2 in zip(str1, str2))
    

In [10]:
def extendable(node, tree, ith_seq, _2d):
    queue = deque()
    if hamming_distance(tree.root.seq, node.seq) <= _2d:
        queue.append(tree.root)
        # print("Root:", tree.root.seq)
        # print("Node to add: ", node.seq, " from sequence ", ith_seq)
        iteration = 0
        while len(queue) > 0 and queue[0].depth < ith_seq - 3:
            
            # print("Queue contents at iteration ", iteration, ":")
            # for i in queue:
            #     print(i.seq)
            
            # print("while loop check")
            front = queue.popleft()
            for child in front.children:
                if hamming_distance(child.seq, node.seq) <= _2d:
                    queue.append(child)
            
            iteration += 1
        
        
    # print("Queue contents returned:")
    # for i in queue:
    #     print(i.seq)

    return queue
        

In [11]:
def prune_branches(tree):
    for leaf in tree.current_leaves:
        if leaf.depth < tree.height:
            current_node = leaf
            num_siblings = len(current_node.parent.children) - 1
            branch_list = [leaf.seq]
            while num_siblings == 0:
                current_node = current_node.parent
                num_siblings = len(current_node.parent.children) - 1
                branch_list.append(current_node.seq)
            parent = current_node.parent
            parent.remove_child(current_node)
            tree.unsave_leaf(leaf)
            

In [12]:
def find_paths(node, path, paths):
    if node is None:
        return
    path.append(node)
    if not node.children:
        clique = Clique(path)
        paths.append(clique)
    
    for child in node.children:
        branch_path = copy.deepcopy(path)
        find_paths(child, branch_path, paths)
        
    
    
    return paths

In [13]:
def merge(new_clique, merged_cliques, _2d):
    new_cliques = []
    for prev_clique in merged_cliques:
        
        match = True
        for seq_1 in prev_clique.seqs:
            if not match:
                break
            for seq_2 in new_clique.seqs:
                if hamming_distance(seq_1, seq_2) > _2d:
                    match = False
                    break

        if match:
            merged_clique = copy.deepcopy(prev_clique)
            for node in new_clique.nodes:
                if node.seq not in prev_clique.seqs:
                    merged_clique.nodes.append(node)
                    merged_clique.seqs.append(node.seq)
            new_cliques.append(merged_clique)
    new_cliques.append(new_clique)
    merged_cliques.extend(new_cliques)

In [14]:
def score_motifs(motifs, motif_length):
    scores = []
    for motif in motifs:
        
        profile = [[0] * motif_length for _ in range(4)]
        indices = {
            'A': 0,
            'C': 1,
            'G': 2,
            'T': 3,
            0: 'A',
            1: 'C',
            2: 'G',
            3: 'T'
        }
        for i in range(len(motif.seqs)):
            for j in range(motif_length):
                profile[indices[motif.seqs[i][j]]][j] += 1
                
        consensus = ""
                
        for j in range(motif_length):
            max_base = max(row[j] for row in profile)
            
            max_base_index = next(idx for idx, row in enumerate(profile) if row[j] == max_base)
            consensus += indices[max_base_index]
            
        score = 0
        for seq in motif.seqs:
            score += hamming_distance(seq, consensus)
        
        motif.consensus = consensus
        motif.score = score
        scores.append((score, motif))
    
    sorted_scores = sorted(scores, key=lambda x: x[0])
    
    for i in range(len(sorted_scores)):
        sorted_scores[i][1].rank = i+1
        
            
        

### STEP 1: NODE SELECTION ###

In [15]:
def select_nodes(sequence_list, l, _2d):
        
    num_seq = len(sequence_list)
        
    reference_seq_1 = sequence_list[0]
    reference_seq_2 = sequence_list[1]
    reference_pair_set = set()
    reference_pair_list = []
    reference_pair_counter = 0
        
    for i in range(len(reference_seq_1) - l + 1):
        for j in range(len(reference_seq_2) - l + 1):
            if hamming_distance(reference_seq_1[i:i+l], reference_seq_2[j:j+l]) <= _2d:
                ref_sub_seq = reference_seq_1[i:i+l]
                ref_sub_seq2 = reference_seq_2[j:j+l]
                if (ref_sub_seq, ref_sub_seq2) in reference_pair_set:
                    break
                reference_pair = Reference_Pair(pair_id=reference_pair_counter, seq1=ref_sub_seq, seq2=ref_sub_seq2, pos1=i, pos2=j, num_seq=num_seq)
                
                # print("refpair candidate: ", ref_sub_seq, ref_sub_seq2)
    
                
                count_seqs_with_nodes = 0
                for k in range(2, num_seq):                
                    has_nodes = False
                    for m in range(len(sequence_list[k]) - l + 1):
                        sub_seq = sequence_list[k][m:m+l]
                        if hamming_distance(sub_seq, ref_sub_seq) <= _2d and hamming_distance(sub_seq, ref_sub_seq2) <= _2d:
                            seq_node = Sequence_Node(seq=sub_seq, id=k, pos=m)
                            reference_pair.node_sets[k-2].add_member(seq_node)
                            has_nodes = True
                    if has_nodes:
                        count_seqs_with_nodes += 1
                
                if count_seqs_with_nodes == num_seq - 2:
                    reference_pair_list.append(reference_pair)
                    reference_pair_set.add((ref_sub_seq, ref_sub_seq2))
                    reference_pair_counter += 1
                    # print("Reference Pair ", reference_pair.pair_id)
                    
    return reference_pair_list
                

            




### TEST NODE SELECTION ###

### STEP 2: TREE CONSTRUCTION ###

In [16]:

def construct_trees(reference_pair_list, _2d):
    
    num_seq = len(reference_pair_list[0].node_sets) + 2
    tree_count = 0
    merged_cliques = []
    for pair in reference_pair_list:
        node_sets = pair.node_sets
        for root in node_sets[0].members:
            root_node = Tree_Node(seq=root.seq, seq_id=root.id, pos=root.pos)
            tree = Tree(root_node)
            
            for i in range(3, num_seq):
                flag = False
                for node in node_sets[i-2].members:
                    branches = extendable(node, tree, i, _2d)
                    if branches:
                        # print("branches check")
                        for branch in branches:
                            new_leaf = Tree_Node(seq=node.seq, seq_id=node.id, pos=node.pos)
                            branch.add_child(new_leaf)
                            tree.save_leaf(new_leaf)
                            tree.unsave_leaf(branch)
                        flag = True
                if not flag:
                    tree = None
                    break
                tree.height += 1
                prune_branches(tree)
            if tree:    
                pair.tree_list.append(tree)
                tree.id = tree_count
                tree_count += 1
                
                new_cliques = find_paths(tree.root, [], [])

                for new_clique in new_cliques:
                    
                    new_clique.nodes.insert(0, Tree_Node(pair.seq2, 1, pair.pos2))
                    new_clique.seqs.insert(0, pair.seq2)
                    
                    new_clique.nodes.insert(0, Tree_Node(pair.seq1, 0, pair.pos1))
                    new_clique.seqs.insert(0, pair.seq1)

                    merge(new_clique, merged_cliques, _2d)
                    
                    
    return merged_cliques


In [17]:
num_procs = cpu_count()
print(num_procs)

8


### TEST TREE CONSTRUCTION ###

### PYTREEMOTIF ###

In [18]:
def find_motif(DNA, motif_length, d=1):
    
    _2d = d * 2
    
    reference_pairs = select_nodes(DNA, motif_length, _2d)
    
    if not reference_pairs:
        return "no reference pairs found"
    
    motifs = construct_trees(reference_pairs, _2d)
    
    score_motifs(motifs, motif_length)
    
    best_motif = None
    for motif in motifs:
        if motif.rank == 1:
            best_motif = motif
    
    print("Best Motif: Motif " + str(motifs.index(best_motif)+1) + ", " + str(best_motif.consensus))
    print()
        
    
    for i in range(len(motifs)):
        print("Motif " + str(i+1) + ", Consensus: " + motifs[i].consensus + ", Rank: " + str(motifs[i].rank) + ", Score: " + str(motifs[i].score))
        for node in motifs[i].nodes:
            print("Seq: " + str(node.seq_id) + ", Motif Position: " + str(node.pos) + ", String: " + str(node.seq))
        print()
        
    return best_motif.consensus
    
    

### TESTING ###

In [19]:
test1 = read_file_to_list("data1.txt")

find_motif(test1, 10)

Best Motif: Motif 3, GGGTCTAAGC

Motif 1, Consensus: AGGGGTCTAA, Rank: 3, Score: 20
Seq: 0, Motif Position: 65, String: TCGGGTCTAA
Seq: 1, Motif Position: 32, String: GAGGGTCTAA
Seq: 2, Motif Position: 10, String: GGGGGTCTAA
Seq: 3, Motif Position: 6, String: AGGGGTCTAA
Seq: 4, Motif Position: 12, String: TTGGGTCTAA
Seq: 5, Motif Position: 88, String: CGGGGTCTAA
Seq: 6, Motif Position: 49, String: CCGGGTCTAA
Seq: 7, Motif Position: 73, String: TAGGGTCTAA
Seq: 8, Motif Position: 54, String: TCGGGTCTAA
Seq: 9, Motif Position: 81, String: ACGGGTCTAA
Seq: 10, Motif Position: 70, String: GGGGGTCTAA
Seq: 11, Motif Position: 53, String: CTGGGTCTAA
Seq: 12, Motif Position: 32, String: AGGGGTCTAA
Seq: 13, Motif Position: 60, String: AGGGGTCTAA
Seq: 14, Motif Position: 42, String: CCGGGTCTAA

Motif 2, Consensus: GGGGTCTAAG, Rank: 2, Score: 9
Seq: 0, Motif Position: 66, String: CGGGTCTAAG
Seq: 1, Motif Position: 33, String: AGGGTCTAAG
Seq: 2, Motif Position: 11, String: GGGGTCTAAG
Seq: 3, Motif P

'GGGTCTAAGC'

In [20]:
test2 = read_file_to_list("data2.txt")

find_motif(test2, 10)

Best Motif: Motif 1, GGGTCTAAGC

Motif 1, Consensus: GGGTCTAAGC, Rank: 1, Score: 14
Seq: 0, Motif Position: 47, String: GGGTCTAAGG
Seq: 1, Motif Position: 41, String: GGGCCTAAGC
Seq: 2, Motif Position: 41, String: GGATCTAAGC
Seq: 3, Motif Position: 42, String: GGGTCTAACC
Seq: 4, Motif Position: 17, String: GGGTCTAAGC
Seq: 5, Motif Position: 60, String: GGGTCTAGGC
Seq: 6, Motif Position: 37, String: GGGTCGAAGC
Seq: 7, Motif Position: 18, String: GGGTCTAATC
Seq: 8, Motif Position: 43, String: GGGTCTGAGC
Seq: 9, Motif Position: 17, String: GGATCTAAGC
Seq: 10, Motif Position: 8, String: GGGTCAAAGC
Seq: 11, Motif Position: 83, String: AGGTCTAAGC
Seq: 12, Motif Position: 33, String: GGGTCGAAGC
Seq: 13, Motif Position: 20, String: GGGGCTAAGC
Seq: 14, Motif Position: 84, String: GGGTCCAAGC



'GGGTCTAAGC'

### 15,4 BENCHMARKING ###

In [21]:
def generate_15_4_dataset(seq_len):
    bases = ['A', 'C', 'G', 'T']
    
    motif = ''.join(random.choice(bases) for _ in range(15))
    
    print("Implanted motif: " + motif)
    print()
    
    dataset = []
    implanted_motifs = []
    
    for i in range(5):
        input_seq = ''.join(random.choice(bases) for _ in range(seq_len))
        
        sub_positions = [random.randint(0, 14) for _ in range(4)]
        
        implanted_motif = motif
        
        for position in sub_positions:
            mutation = random.choice(bases)
            implanted_motif = implanted_motif[:position] + mutation + implanted_motif[position + 1:]
            
        implant_position = random.randint(0, seq_len - 1)
        
        implanted_motifs.append((implanted_motif, implant_position))
            
        input_seq_w_motif = input_seq[:implant_position] + implanted_motif + input_seq[implant_position + 15:]
        
        dataset.append(input_seq_w_motif)
    
    for im in implanted_motifs:
        print(im[0], im[1])
        
    return dataset, motif
        


In [22]:
dataset, implanted_motif = generate_15_4_dataset(200)

Implanted motif: GTGCTTCATCGAGCT

GTCCTTCATCGAGGT 181
GTGCTTCGTTGCGCT 2
GTTCTTTATAGAGCT 28
GTGCTTCATCAAGCT 31
GTGATTCATCTAGCC 32


In [23]:
found_motif = find_motif(dataset, 15, 4)


Best Motif: Motif 796, GTGCTTCATCGAGCT

Motif 1, Consensus: CAGCCATAACTGATG, Rank: 89, Score: 21
Seq: 0, Motif Position: 1, String: CAGGCGTAAAGGATT
Seq: 1, Motif Position: 117, String: CAGCCATAACTAACT
Seq: 2, Motif Position: 88, String: TATCCACAAATAATG
Seq: 3, Motif Position: 147, String: AAGCCAACACTGATG
Seq: 4, Motif Position: 112, String: GAACCGTTACTGCTG

Motif 2, Consensus: AGCCATAACTGATGA, Rank: 90, Score: 21
Seq: 0, Motif Position: 2, String: AGGCGTAAAGGATTA
Seq: 1, Motif Position: 118, String: AGCCATAACTAACTT
Seq: 2, Motif Position: 89, String: ATCCACAAATAATGT
Seq: 3, Motif Position: 148, String: AGCCAACACTGATGA
Seq: 4, Motif Position: 113, String: AACCGTTACTGCTGG

Motif 3, Consensus: GATAAAGGATCAGGT, Rank: 566, Score: 25
Seq: 0, Motif Position: 5, String: CGTAAAGGATTACGT
Seq: 1, Motif Position: 89, String: ACGAAAGGCGCCGGT
Seq: 2, Motif Position: 58, String: GCTCAAGGGTCGTGC
Seq: 3, Motif Position: 104, String: GATGAAAGAGTGGGT
Seq: 4, Motif Position: 47, String: GAGATAAGGTCAGGT

M

In [24]:
assert found_motif == implanted_motif


### PARALLELISM TESTING ###

In [25]:
def build_trees(pair, _2d):
    new_cliques = []
    
    
    node_sets = pair.node_sets
    print("Ref pair " + str(pair.pair_id) + ": " + str(len(node_sets[0].members)) + " roots")
    
    num_seq = len(node_sets) + 2
    for root in node_sets[0].members:
        root_node = Tree_Node(seq=root.seq, seq_id=root.id, pos=root.pos)
        tree = Tree(root_node)
        
        for i in range(3, num_seq):
            flag = False
            for node in node_sets[i-2].members:
                branches = extendable(node, tree, i, _2d)
                if branches:
                    # print("branches check")
                    for branch in branches:
                        new_leaf = Tree_Node(seq=node.seq, seq_id=node.id, pos=node.pos)
                        branch.add_child(new_leaf)
                        tree.save_leaf(new_leaf)
                        tree.unsave_leaf(branch)
                    flag = True
            if not flag:
                tree = None
                break
            tree.height += 1
            prune_branches(tree)
        if tree:    
            pair.tree_list.append(tree)
            # tree.id = tree_count
            # tree_count += 1
            
            new_cliques = find_paths(tree.root, [], [])

            for new_clique in new_cliques:
                
                new_clique.nodes.insert(0, Tree_Node(pair.seq2, 1, pair.pos2))
                new_clique.seqs.insert(0, pair.seq2)
                
                new_clique.nodes.insert(0, Tree_Node(pair.seq1, 0, pair.pos1))
                new_clique.seqs.insert(0, pair.seq1)     
                
    return new_cliques  

In [26]:

def parallel_construct_trees(reference_pair_list, _2d):
    
    num_seq = len(reference_pair_list[0].node_sets) + 2
    unmerged_clique_lists = []
    merged_cliques = []
    
    
    num_processes = cpu_count()
    with Pool(num_processes) as pool:
        unmerged_clique_lists = pool.starmap(build_trees, [(pair, _2d) for pair in reference_pair_list])
    
    unmerged_cliques = [clique for sublist in unmerged_clique_lists for clique in sublist]
    
    for unmerged_clique in unmerged_cliques:
        merge(unmerged_clique, merged_cliques, _2d)


    return merged_cliques

In [27]:
def parallel_find_motif(DNA, motif_length, d=1):
    
    _2d = d * 2
    
    reference_pairs = select_nodes(DNA, motif_length, _2d)
    
    if not reference_pairs:
        return "no reference pairs found"
    
    motifs = parallel_construct_trees(reference_pairs, _2d)
    
    score_motifs(motifs, motif_length)
    
    best_motif = None
    for motif in motifs:
        if motif.rank == 1:
            best_motif = motif
    
    print("Best Motif: Motif " + str(motifs.index(best_motif)+1) + ", " + str(best_motif.consensus))
    print()
        
    
    for i in range(len(motifs)):
        print("Motif " + str(i+1) + ", Consensus: " + motifs[i].consensus + ", Rank: " + str(motifs[i].rank) + ", Score: " + str(motifs[i].score))
        for node in motifs[i].nodes:
            print("Seq: " + str(node.seq_id) + ", Motif Position: " + str(node.pos) + ", String: " + str(node.seq))
        print()
        
    return best_motif.consensus

In [28]:
parallel_found_motif = parallel_find_motif(dataset, 15, 4)


Ref pair 0: 2 rootsRef pair 36: 1 rootsRef pair 72: 2 rootsRef pair 108: 2 roots


Ref pair 144: 2 roots
Ref pair 180: 2 rootsRef pair 73: 2 rootsRef pair 1: 3 rootsRef pair 37: 1 roots


Ref pair 109: 1 roots

Ref pair 145: 4 roots
Ref pair 181: 1 rootsRef pair 216: 5 rootsRef pair 252: 2 roots
Ref pair 2: 2 roots

Ref pair 74: 4 rootsRef pair 38: 1 roots

Ref pair 182: 1 roots

Ref pair 3: 2 rootsRef pair 146: 3 rootsRef pair 110: 1 rootsRef pair 217: 1 roots
Ref pair 39: 1 rootsRef pair 253: 2 roots

Ref pair 183: 1 rootsRef pair 75: 3 roots




Ref pair 218: 1 rootsRef pair 111: 1 rootsRef pair 147: 1 rootsRef pair 4: 2 rootsRef pair 40: 2 roots

Ref pair 254: 3 roots

Ref pair 76: 3 roots

Ref pair 112: 1 roots
Ref pair 148: 1 rootsRef pair 5: 3 roots
Ref pair 219: 2 rootsRef pair 77: 3 rootsRef pair 41: 1 roots

Ref pair 149: 2 rootsRef pair 220: 2 roots

Ref pair 184: 2 roots
Ref pair 78: 3 roots



Ref pair 42: 3 roots
Ref pair 6: 1 rootsRef pair 255: 4 rootsRef pair 113: 1 roo

In [29]:
large_test, implanted_motif = generate_15_4_dataset(300)

Implanted motif: ATTCACAGACTACTC

ATTCACAGAATTCTC 268
ATTCACAGAATTCTC 172
ATTCATAGACTTCGG 109
ATTCACGGACTACTC 296
ATCCACAGGCTACTC 9


In [30]:
parallel_found_motif2 = parallel_find_motif(large_test, 15, 4)

Ref pair 0: 2 roots
Ref pair 1: 4 rootsRef pair 119: 4 roots
Ref pair 238: 2 roots

Ref pair 120: 2 rootsRef pair 239: 1 rootsRef pair 357: 3 rootsRef pair 2: 4 roots



Ref pair 121: 1 rootsRef pair 240: 1 rootsRef pair 358: 4 roots

Ref pair 3: 4 rootsRef pair 241: 1 roots

Ref pair 476: 1 roots
Ref pair 359: 2 roots
Ref pair 595: 4 rootsRef pair 4: 2 rootsRef pair 122: 3 roots



Ref pair 833: 1 rootsRef pair 5: 6 roots
Ref pair 477: 4 rootsRef pair 242: 3 rootsRef pair 123: 3 rootsRef pair 714: 2 roots

Ref pair 596: 2 roots


Ref pair 834: 2 rootsRef pair 360: 2 rootsRef pair 597: 4 roots
Ref pair 124: 5 rootsRef pair 243: 3 rootsRef pair 478: 1 roots



Ref pair 361: 5 roots
Ref pair 6: 5 rootsRef pair 479: 1 rootsRef pair 715: 8 rootsRef pair 244: 4 roots



Ref pair 598: 2 roots
Ref pair 125: 1 rootsRef pair 480: 2 roots

Ref pair 835: 1 rootsRef pair 362: 2 roots

Ref pair 716: 5 roots
Ref pair 126: 1 rootsRef pair 245: 2 rootsRef pair 7: 3 roots


Ref pair 363: 5 roots
Ref pa

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Ref pair 2180: 2 roots

Ref pair 2064: 2 rootsRef pair 2525: 1 roots



Ref pair 2406: 6 rootsRef pair 2303: 1 roots
Ref pair 1961: 5 roots
Ref pair 2632: 3 rootsRef pair 2181: 1 roots

Ref pair 2407: 4 rootsRef pair 2065: 2 rootsRef pair 2526: 2 roots



Ref pair 1962: 2 rootsRef pair 2633: 2 rootsRef pair 2066: 2 rootsRef pair 2527: 1 roots

Ref pair 2304: 5 roots
Ref pair 2182: 1 roots

Ref pair 2408: 2 roots

Ref pair 1963: 4 rootsRef pair 2067: 1 rootsRef pair 2305: 7 roots
Ref pair 2528: 1 roots
Ref pair 1964: 6 rootsRef pair 2183: 3 roots
Ref pair 2634: 1 roots



Ref pair 2068: 2 rootsRef pair 2737: 1 rootsRef pair 2529: 2 rootsRef pair 2184: 7 roots

Ref pair 2409: 5 roots
Ref pair 2306: 3 rootsRef pair 2635: 4 roots



Ref pair 1965: 4 rootsRef pair 2069: 5 roots

Ref pair 2738: 2 roots

Ref pair 2307: 4 rootsRef pair 2530: 2 rootsRef pair 2070: 3 rootsRef pair 2636: 2 roots
Ref pair 2185: 2 rootsRef pair 2531: 1 roots
Ref pair 2410: 2 roots

Ref pair 2071: 1 roots
Ref pair 2

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)






Ref pair 2971: 2 rootsRef pair 3739: 8 rootsRef pair 3078: 3 roots
Ref pair 3621: 8 rootsRef pair 3187: 1 roots


Ref pair 3296: 1 roots

Ref pair 3188: 2 roots
Ref pair 2972: 1 rootsRef pair 3079: 4 rootsRef pair 3491: 2 rootsRef pair 3189: 2 rootsRef pair 3740: 6 roots


Ref pair 3297: 1 roots



Ref pair 2973: 3 rootsRef pair 3298: 3 roots

Ref pair 3299: 2 rootsRef pair 3741: 11 rootsRef pair 2974: 4 rootsRef pair 3080: 1 rootsRef pair 3190: 5 roots


Ref pair 3492: 1 rootsRef pair 3081: 1 rootsRef pair 3300: 3 rootsRef pair 3622: 14 roots
Ref pair 3742: 5 roots



Ref pair 3191: 2 roots
Ref pair 3082: 1 rootsRef pair 3494: 4 rootsRef pair 3493: 1 roots


Ref pair 3623: 4 rootsRef pair 3743: 2 rootsRef pair 3301: 1 roots


Ref pair 3744: 1 roots
Ref pair 3192: 3 roots

Ref pair 3624: 5 rootsRef pair 3302: 1 rootsRef pair 3745: 5 rootsRef pair 3495: 2 roots
Ref pair 3083: 3 roots


Ref pair 3193: 2 roots
Ref pair 3625: 4 rootsRef pair 3303: 2 roots

Ref pair 3084: 3 roots
Ref pai